# 2SAT Problem

In this assignment you will implement one or more algorithms for the 2SAT problem. Here are 6 different 2SAT instances:

The file format is as follows. In each instance, the number of variables and the number of clauses is the same, and this number is specified on the first line of the file. Each subsequent line specifies a clause via its two literals, with a number denoting the variable and a "-" sign denoting logical "not". For example, the second line of the first data file is "-16808 75250", which indicates the clause ¬x16808∨x75250.

Your task is to determine which of the 6 instances are satisfiable, and which are unsatisfiable. In the box below, enter a 6-bit string, where the ith bit should be 1 if the ith instance is satisfiable, and 0 otherwise. For example, if you think that the first 3 instances are satisfiable and the last 3 are not, then you should enter the string 111000 in the box below.

DISCUSSION: This assignment is deliberately open-ended, and you can implement whichever 2SAT algorithm you want. For example, 2SAT reduces to computing the strongly connected components of a suitable graph (with two vertices per variable and two directed edges per clause, you should think through the details). This might be an especially attractive option for those of you who coded up an SCC algorithm in Part 2 of this specialization. Alternatively, you can use Papadimitriou's randomized local search algorithm. (The algorithm from lecture is probably too slow as stated, so you might want to make one or more simple modifications to it --- even if this means breaking the analysis given in lecture --- to ensure that it runs in a reasonable amount of time.) A third approach is via backtracking. In lecture we mentioned this approach only in passing; see Chapter 9 of the Dasgupta-Papadimitriou-Vazirani book, for example, for more details

20
n= 32
graph #items: 25
vertices idx {'27', '13', '24', '11', '6', '10', '31', '9', '22', '14', '26', '8', '4', '1', '15', '16', '3', '19', '21', '25', '17', '5', '7', '12', '2', '20', '28', '29', '30', '32', '23', '18'}
idx_to_v {'23': '-6', '28': '-8', '29': '-18', '27': '8', '15': '12', '30': '18', '13': '-12', '11': '5', '16': '-2', '31': '-15', '19': '13', '21': '19', '25': '16', '5': '-9', '17': '-13', '6': '-20', '10': '4', '7': '9', '12': '-4', '9': '-5', '2': '-10', '20': '17', '14': '2', '24': '6', '26': '-16', '32': '15', '8': '20', '22': '-19', '3': '3', '4': '10', '1': '-3', '18': '-17'}
v_to_idx {'-2': '16', '-9': '5', '-12': '13', '15': '32', '-6': '23', '13': '19', '-4': '12', '18': '30', '16': '25', '-17': '18', '-13': '17', '19': '21', '-8': '28', '10': '4', '17': '20', '6': '24', '5': '11', '-10': '2', '9': '7', '2': '14', '20': '8', '-15': '31', '-18': '29', '12': '15', '-5': '9', '-3': '1', '-19': '22', '-20': '6', '4': '10', '-16': '26', '3': '3', '8': '27'}
gra

In [19]:
def reverse_graph(graph):
    rev = {}
    
    for v, k in graph.items():
        for i in k:
            rev.setdefault(i, [])
            rev[i].append(v)
            
    return rev

class dfs_obj(object):
    UNEXPLORED = 0
    EXPLORING = 1
    EXPLORED = 2


    def __init__(self, graph, rev_graph, vertices):
        
        self.graph = graph
        self.rev_graph = rev_graph
        self.vertices = vertices
        
        self.state = {}
        self.leaders = {}
        self.finish_time = {} #old label -> new label
        self.finish_time_rev = {} #new label -> old label


        self.t = 0
        self.s = None
        
    def init_state(self):
        state = {}
        for k in self.vertices:
            state[k] = dfs_obj.UNEXPLORED
            
        return state

    """
        ITERATIVE IMPLEMENTATION        
    """ 
    def DFS_ITERATIVE_1(self, i):

        #print ("node {} explored".format(i))
        #self.leaders[i] = self.s        

        stack = []
        stack.append(i)

        while (stack):
            v = stack.pop() # remove last

            if self.state[v] == dfs_obj.EXPLORING: 
                self.state[v] = dfs_obj.EXPLORED
                self.t += 1
                self.finish_time[v] = str(self.t)

            if self.state[v] == dfs_obj.UNEXPLORED:
                self.state[v] = dfs_obj.EXPLORING            
                stack.append(v)

                if v in self.rev_graph:
                    for j in self.rev_graph[v]:
                        if self.state[j] == dfs_obj.UNEXPLORED:
                            stack.append(j)                        

    def DFS_ITERATIVE_LOOP1(self):
        n = len(self.vertices)
    
        for i in range(n, 0, -1):
            i = str(i)
            
            if self.state[i] != dfs_obj.EXPLORED:
                #print("loop1 explore", i)
                self.s = i
                self.DFS_ITERATIVE_1(i)
       
    
    def DFS_ITERATIVE_2(self,i):
        # i is the new label
        
        stack = []
        stack.append(i)

        while (stack):
            v = stack.pop() # remove last

            if self.state[v] == dfs_obj.EXPLORING: 
                self.state[v] = dfs_obj.EXPLORED
                
                # leaders refer to old label
                self.leaders[self.finish_time_rev[v]] = self.finish_time_rev[self.s]


            if self.state[v] == dfs_obj.UNEXPLORED:
                self.state[v] = dfs_obj.EXPLORING            
                stack.append(v)

                if v in self.finish_time_rev and self.finish_time_rev[v] in self.graph:
                    for j in self.graph[self.finish_time_rev[v]]:
                        j = self.finish_time[j]

                        if self.state[j] == dfs_obj.UNEXPLORED:
                            stack.append(j)                        


    def DFS_ITERATIVE_LOOP2(self):
        n = len(self.vertices)
    
        for i in range(n, 0, -1):
            # iterate of the *new* labels
            i = str(i)
            
            if self.state[i] != dfs_obj.EXPLORED:
                self.s = i
                self.DFS_ITERATIVE_2(i)
    """
        RECURSIVE IMPLEMENTATION        
    """            
    def DFS1(self, i):

        # Set Explored 
        self.state[i] = 1
        
        #print ("node {} explored".format(i))        
        if i in self.rev_graph:
            for j in self.rev_graph[i]:
                if self.state[j] == 0:
                    self.DFS1(j)

        self.t += 1
        self.finish_time[i] = str(self.t)
        

    def DFS_LOOP1(self):
        self.t = 0
        n = len(self.vertices)
    
        for i in range(n, 0, -1):
            i = str(i)
            
            if self.state[i] == 0:
                self.DFS1(i)

                
    def DFS2(self,i):

        self.state[i] = 1
        #print ("node {} explored".format(i))
        
        self.leaders[self.finish_time_rev[i]] = self.finish_time_rev[self.s]

        for j in self.graph[self.finish_time_rev[i]]:
            #print (self.finish_time)
            j = self.finish_time[j]
            
            if self.state[j] == 0:
                self.DFS2(j)


    def DFS_LOOP2(self):
        n = len(self.vertices)
    
        for i in range(n, 0, -1):
            # iterate of the *new* labels
            i = str(i)
            
            if self.state[i] == 0:
                self.s = i
                self.DFS2(i)

    def run_DFS_LOOP(self):
        print (self.graph, self.rev_graph)
        print("step 1")
        self.state = self.init_state()
        self.DFS_LOOP1()

        print ("finish_time", self.finish_time)
        
        print ("step 2")
        self.finish_time_rev = {str(d):str(k) for k,d in self.finish_time.items()}

        self.state = self.init_state()
#         print (self.finish_time_rev)
        self.DFS_LOOP2()
    
    def run_DFS_LOOP_ITERATIVE(self):
        #print("step 1")
        self.state = self.init_state()
        
        self.DFS_ITERATIVE_LOOP1()
        
        #print ("step 2")
        self.finish_time_rev = {str(d):str(k) for k,d in self.finish_time.items()}

        self.state = self.init_state()

#         print (self.finish_time_rev)
        self.DFS_ITERATIVE_LOOP2()



In [39]:
# Use SCC and verify than no Connected Component contains itself and its negation
import collections

FILES = ["input_beaunus_10_20.txt", 
         "input_beaunus_11_40.txt",
         "2sat1.txt", 
         "2sat2.txt", 
         "2sat3.txt", 
         "2sat4.txt",
         "2sat5.txt",
         "2sat6.txt"]

expected = [False,
            True,
            True,
            False,
            True,
            True,
            False,
            False]

for f in range(len(FILES)):

    fp = open(FILES[f], 'r')
    data = fp.readlines()

    n = int(data[0])
    graph = collections.defaultdict(list)
    vertices = set()
    v_to_idx = {}
    idx_to_v = {}
    i = 1
    def get_idx(v):
        # Generate a unique id for a node if it hasn't been seen before
        global i
        if v in v_to_idx:
            return v_to_idx[v]
        else:
            v_to_idx[v] = str(i)
            idx_to_v[str(i)] = v
            i += 1

            return v_to_idx[v]

    # 1. Create Graph
    for row in data[1:]:
        v1, v2 = row.strip().split(" ")

        notv1 = str(-1*int(v1))
        notv2 = str(-1*int(v2))

        v1 = get_idx(v1)
        v2 = get_idx(v2)
        notv1 = get_idx(notv1)
        notv2 = get_idx(notv2)

        vertices.add(v1)
        vertices.add(v2)
        vertices.add(notv1)
        vertices.add(notv2)

        graph[notv1].append(v2)
        graph[notv2].append(v1)

    # 2. Run SCC
    rev_graph = reverse_graph(graph)
    g = dfs_obj(graph, rev_graph, vertices)
    g.run_DFS_LOOP_ITERATIVE()

    def check_satisfiable(leaders):
        leader_list = collections.defaultdict(list)
        for l in leaders:
            leader_list[int(idx_to_v[leaders[l]])].append(int(idx_to_v[l]))

        # Check whether a node and its complement exist in the same Connected Component
        for l in leader_list:
            items = leader_list[l]
            for i in range(0,len(items)-1):
                for j in range(i,len(items)):
                    if items[i] == -items[j]:
                        print ("Not Satisfiable", items[i])
                        return False                
        return True

    # 3. Test Satisfiability
    sat = check_satisfiable(g.leaders)
    print ("satisfiable", sat) 
    assert sat == expected[f]

print "PASS"

20
Not Satisfiable 8
satisfiable False
40
satisfiable True
100000
satisfiable True
200000
Not Satisfiable -162741
satisfiable False
400000
satisfiable True
600000
satisfiable True
800000
Not Satisfiable -618692
satisfiable False
1000000
Not Satisfiable 410420
satisfiable False
